In [80]:
import pandas as pd
import trigrid
import matplotlib.pyplot as plt
import numpy as np
from scipy import spatial
from math import sqrt, degrees

In [81]:
data = pd.read_hdf("/media/data/Data/BallBearing/HIPS/2,45mm_base_cyan.hdf5")

In [82]:
tracked = data.loc[0, ['x', 'y']].values

In [87]:
%matplotlib qt
plt.plot(*tracked.T, 'o')

In [38]:
tree = spatial.KDTree(tracked)
dists, indxs = tree.query(tracked, 2)
dists = dists[:, 1]
indxs = indxs[:, 1]
median = np.median(dists)

In [39]:
vecs = tracked - tracked[indxs, :]

In [40]:
vecs

array([[ 11.0407104 , -19.96859458],
       [ 10.74961342, -22.7334786 ],
       [ 25.18155385,  -0.94221183],
       ...,
       [ -8.66687269,  -0.62989276],
       [  8.66687269,   0.62989276],
       [-16.10728436,  -0.17947902]])

In [41]:
angles = np.arctan2(vecs[:, 1], vecs[:, 0])

In [42]:
angles_threshold = angles[(angles<0.5)*(angles>-0.5)]

In [43]:
median_angle = np.median(angles_threshold)

In [44]:
import cv2

In [45]:
hull = cv2.convexHull(tracked.astype(np.float32)).squeeze().T

In [91]:
new_grid = trigrid.grid(median, 2000, degrees(median_angle)+30, hull.T)

In [92]:
plt.plot(*new_grid, 'x')
# plt.plot(*tracked.T, 'o')

In [ ]:
from scipy import optimize

In [66]:
tree = spatial.KDTree(tracked)
def dists_to_tracked(x0):
    newgrid = trigrid.grid(median, 2000, degrees(median_angle)+60, hull.T)
    newgrid[0, :] += x0
    dists, indxs = tree.query(newgrid.T, 1)
    return np.sum(dists)
    

In [67]:
x1 = optimize.minimize(dists_to_tracked, 0)

In [70]:
x1.x[0]

0.42570102004619603

In [71]:
def minimise_y(y0):
    newgrid = trigrid.grid(median, 2000, degrees(median_angle)+60, hull.T)
    newgrid[0, :] += x1.x[0]
    newgrid[1, :] += y0
    dists, indxs = tree.query(newgrid.T, 1)
    return np.sum(dists)
    

In [72]:
y1 = optimize.minimize(minimise_y, 0)

In [73]:
y1.x[0]

4.8743129691532685

In [78]:
newgrid = trigrid.grid(median, 2000, degrees(median_angle)+30, hull.T)
# newgrid[0, :] += x1.x[0]
# newgrid[1, :] += y1.x[0]

In [93]:
plt.plot(*new_grid, 'x')
plt.plot(*tracked.T, 'o')